In [ ]:
!git clone https://github.com/muxspace/facial_expressions.git

Cloning into 'facial_expressions'...
remote: Enumerating objects: 14214, done.
remote: Total 14214 (delta 0), reused 0 (delta 0), pack-reused 14214
Receiving objects: 100% (14214/14214), 239.65 MiB | 35.48 MiB/s, done.
Resolving deltas: 100% (223/223), done.
Checking out files: 100% (13996/13996), done.


In [ ]:
import csv
data={}
with open('/content/facial_expressions/data/legend.csv') as f:
  reader=csv.reader(f)
  next(reader)
  for row in reader:
    key=row[2].lower()
    if key in data:
      data[key].append(row[1])
    else:
      data[key]=[row[1]]


In [ ]:
emotion_list=list(data.keys())

In [ ]:
import os
os.mkdir('/content/drive/MyDrive/M.L projects/facial_recognisation/master_data')
os.mkdir('/content/drive/MyDrive/M.L projects/facial_recognisation/master_data/training')
os.mkdir('/content/drive/MyDrive/M.L projects/facial_recognisation/master_data/testing')


In [ ]:
for emotions in emotion_list:
  os.mkdir(os.path.join('/content/drive/MyDrive/M.L projects/facial_recognisation/master_data/training/',emotions))
  os.mkdir(os.path.join('/content/drive/MyDrive/M.L projects/facial_recognisation/master_data/testing/',emotions))

In [ ]:
from shutil import copyfile
split_size=0.8
for emotion,images in data.items():
  train_size=int(split_size*len(images))
  train_images=images[:train_size]
  test_images=images[train_size:]
  for image in train_images:
    source=os.path.join('/content/facial_expressions/images',image)
    des=os.path.join('/content/drive/MyDrive/M.L projects/facial_recognisation/master_data/training',emotion,image)
    copyfile(source,des)
  for image in test_images:
    source=os.path.join('/content/facial_expressions/images',image)
    des=os.path.join('/content/drive/MyDrive/M.L projects/facial_recognisation/master_data/testing',emotion,image)
    copyfile(source,des)


In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import MaxPooling2D,Dense,Flatten,Conv2D

In [2]:
model=tf.keras.models.Sequential([
                                 Conv2D(16,(3,3),activation='relu',input_shape=(100,100,3)),
                                 MaxPooling2D(2,2),
                                 Conv2D(32,(3,3),activation='relu'),
                                 MaxPooling2D(2,2),
                                 Conv2D(64,(3,3),activation='relu'),
                                 MaxPooling2D(2,2),
                                 Flatten(),
                                 Dense(512,activation='relu'),
                                 Dense(8,activation='softmax')
])

In [3]:
model.compile(optimizer=Adam(lr=0.01),loss='categorical_crossentropy',metrics=['acc'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 98, 98, 16)        448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 49, 49, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 47, 47, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 23, 23, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 21, 21, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 10, 10, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 6400)              0

In [5]:
train_dir='/content/drive/MyDrive/M.L projects/facial_recognisation/master_data/training'
test_dir='/content/drive/MyDrive/M.L projects/facial_recognisation/master_data/testing'

In [6]:
train_datagen=ImageDataGenerator(rescale=1.0/255)
train_generator=train_datagen.flow_from_directory(
    train_dir,
    target_size=(100,100),
    class_mode='categorical',
    batch_size=128
)
test_datagen=ImageDataGenerator(rescale=1.0/255)
test_generator=test_datagen.flow_from_directory(
    test_dir,
    target_size=(100,100),
    class_mode='categorical',
    batch_size=500
)

Found 10941 images belonging to 8 classes.
Found 2742 images belonging to 8 classes.


In [7]:
es=EarlyStopping(monitor='val_acc',patience=5,min_delta=0.01)

In [8]:
model.fit_generator(train_generator,
                    epochs=20,
                    verbose=1,
                    validation_data=test_generator,
                    callbacks=[es])

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
28/86 [========>.....................] - ETA: 27:16 - loss: 2.1407 - acc: 0.4796

KeyboardInterrupt: ignored